# Stock Agent

In [64]:
# Basic Agent

In [65]:
import yfinance as yf
from IPython.display import display, Markdown
from langchain.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, ToolMessage

In [66]:
# Load AI model from Ollama
llm = ChatOllama(model="mistral")

## Tools

In [67]:
@tool
def get_stock_info(ticker: str) -> dict:
    """
    Fetch authoritative stock fundamentals and identity information
    using Yahoo Finance.

    Use this tool to:
    - Validate a ticker symbol
    - Resolve the exact company and share class
    - Retrieve factual fundamentals and pricing

    Returns:
        dict with:
            - requested_ticker
            - resolved_ticker
            - company_name
            - current_price
            - market_cap
            - trailing_pe
            - eps
            - dividend_yield
    """
    try:
        requested = ticker.strip().upper()
        stock = yf.Ticker(requested)
        info = stock.info

        resolved = info.get("symbol")
        company_name = info.get("longName")

        if not info or not resolved:
            return {
                "status": "error",
                "requested_ticker": requested,
                "error": f"No data found for ticker '{requested}'."
            }

        return {
            "status": "ok",
            "requested_ticker": requested,
            "resolved_ticker": resolved,
            "company_name": company_name,
            "current_price": info.get("currentPrice"),
            "market_cap": info.get("marketCap"),
            "trailing_pe": info.get("trailingPE"),
            "eps": info.get("trailingEps"),
            "dividend_yield": info.get("dividendYield"),
        }

    except Exception as e:
        return {
            "status": "error",
            "requested_ticker": ticker,
            "error": str(e),
        }

In [68]:
@tool
def get_stock_news(ticker: str) -> dict:
    """
    Fetch recent news headlines related to a stock ticker.

    Use this tool when:
    - The user asks about recent events, sentiment, or developments
    - Context beyond fundamentals is required

    Returns:
        dict with:
            - ticker
            - status: ok | no_news_found | error
            - headlines: list of strings
    """
    try:
        ticker = ticker.strip().upper()
        stock = yf.Ticker(ticker)
        news = stock.news

        if not news:
            return {
                "ticker": ticker,
                "status": "no_news_found",
                "headlines": [],
            }

        headlines = [
            item.get("title")
            for item in news[:5]
            if item.get("title")
        ]

        return {
            "ticker": ticker,
            "status": "ok",
            "headlines": headlines,
        }

    except Exception as e:
        return {
            "ticker": ticker,
            "status": "error",
            "error": str(e),
        }

## prompts

In [102]:
system_prompt = """
You are a financial analysis assistant.

Rules:
- Use tools to retrieve stock data.
- Do not invent prices, market caps, or news.
- If a ticker resolves to a different symbol, clearly report it and stop.
- If data is missing, say it is unavailable.
"""

In [97]:
def make_human_prompt(ticker: str) -> str:
    return f"""
You must analyze the stock with ticker symbol: {ticker}.

MANDATORY INSTRUCTIONS:
- Your FIRST action must be to call the tool `get_stock_info`.
- You are NOT allowed to answer directly.
- If you do not call a tool, your response is invalid.
"""

In [98]:
def make_summary_prompt(tool_results: dict) -> str:
    return f"""
You are given factual tool output below.

Rules:
- Use ONLY this data
- Do NOT add anything
- If a value is missing, say "unavailable"

Tool output:
{tool_results}

Produce a clean summary.
"""

## Agent

In [103]:
agent = llm.bind_tools(
    [
        get_stock_info,
        get_stock_news
    ]
)

In [104]:
def run_stock_agent(ticker: str):
    messages = [
        ("system", system_prompt),
        ("human", make_human_prompt(ticker)),
    ]

    # ✅ USE agent, not llm
    ai_msg = agent.invoke(messages)

    if not ai_msg.tool_calls:
        raise RuntimeError("LLM did not call any tools")

    tool_results = {}

    for call in ai_msg.tool_calls:
        tool_name = call["name"]
        tool_args = call["args"]

        result = tool_map[tool_name](**tool_args)
        tool_results[tool_name] = result

        if tool_name == "get_stock_info":
            if (
                result.get("status") == "ok"
                and result["requested_ticker"] != result["resolved_ticker"]
            ):
                return {
                    "status": "needs_confirmation",
                    "message": "Ticker mismatch detected"
                }

    summary_prompt = make_summary_prompt(tool_results)

    final_msg = agent.invoke([
        ("system", "You are a formatter. Do not add or infer information."),
        ("human", summary_prompt),
    ])

    return {
        "status": "ok",
        "message": final_msg.content,
    }


In [105]:
result = run_stock_agent("GOOG")

display(Markdown(result["message"]))

RuntimeError: LLM did not call any tools

In [87]:
result["message"]

' I have retrieved the following information for ticker symbol GOOG:\n\n1. Company Identity:\n   - Name: Alphabet Inc. Class A Common Stock\n   - Exchange: NASDAQ Global Select Market\n\n2. Key Fundamentals:\n   - Last Price: 1398.07 USD (as of the latest available data)\n   - Market Cap: Not available (last reported market cap was 1,465,977,082,500 USD)\n   - PE Ratio: Not available (last reported PE ratio was 35.76)\n   - Dividend Yield: Not available (Alphabet Inc. does not pay dividends)\n   - Earnings per Share (EPS): Not available (last reported EPS was 29.81 USD)\n\n3. Recent News Status:\n   - I could not retrieve recent news headlines for ticker symbol GOOG at this time.'